In [1]:
import urllib.request, json
import requests
import pandas as pd

import re
import xml.etree.ElementTree as ET

In [2]:
liquidacion_trimestral_uri = 'https://236ws.dpteruel.es/transparencia/dpteruel/wp-content/uploads/sites/3/2021/05/P4400000H-2021-LIQUIDACION-TRIMESTRAL-PRESUPUESTOS-1T.xml'

ns = {
    '': 'http://www.xbrl.org/2003/instance',
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
    'trimloc2017': 'http://www.meh.es/taxonomias/trimloc2017',
    'trimloc2017-econ-ingr': 'http://www.meh.es/taxonomias/trimloc2017-econ-ingr',
    'trimloc2017-resultp': 'http://www.meh.es/taxonomias/trimloc2017-resultp',
    'trimloc2017-exist': 'http://www.meh.es/taxonomias/trimloc2017-exist',
    'xbrldt': 'http://xbrl.org/2005/xbrldt',
    'trimloc2017-ref': 'http://www.meh.es/taxonomias/trimloc2017-ref',
    'trimloc2017-econ-gast-cuentas': 'http://www.meh.es/taxonomias/trimloc2017-econ-gast-cuentas',
    'trimloc2017-prog-cuentas-econ-cruzadas': 'http://www.meh.es/taxonomias/trimloc2017-prog-cuentas-econ-cruzadas',
    'link': 'http://www.xbrl.org/2003/linkbase',
    'trimloc2017-econ-ingr-cuentas': 'http://www.meh.es/taxonomias/trimloc2017-econ-ingr-cuentas',
    'trimloc2017-econ-gast': 'http://www.meh.es/taxonomias/trimloc2017-econ-gast',
    'xlink': 'http://www.w3.org/1999/xlink',
    'trimloc2017-prog': 'http://www.meh.es/taxonomias/trimloc2017-prog',
    'iso4217': 'http://www.xbrl.org/2003/iso4217',
    'trimloc2017-prog-cuentas': 'http://www.meh.es/taxonomias/trimloc2017-prog-cuentas',
    'trimloc2017-anexo5': 'http://www.meh.es/taxonomias/trimloc2017-anexo5',
    'trimloc2017-remt': 'http://www.meh.es/taxonomias/trimloc2017-remt',
    'trimloc2017-anexo4': 'http://www.meh.es/taxonomias/trimloc2017-anexo4',
    'trimloc2017-anexo3': 'http://www.meh.es/taxonomias/trimloc2017-anexo3',
    'xbrldi': 'http://xbrl.org/2006/xbrldi',
    'trimloc2017-anexo2': 'http://www.meh.es/taxonomias/trimloc2017-anexo2',
    'trimloc2017-econ-ingr-importes': 'http://www.meh.es/taxonomias/trimloc2017-econ-ingr-importes',
    'trimloc2017-econ-gast-importes': 'http://www.meh.es/taxonomias/trimloc2017-econ-gast-importes',
    'trimloc2017-anexo1': 'http://www.meh.es/taxonomias/trimloc2017-anexo1',
    'xbrli': 'http://www.xbrl.org/2003/instance'
}
r = requests.get(liquidacion_trimestral_uri)
tree = ET.ElementTree(ET.fromstring(r.text))
root = tree.getroot()

In [3]:
def format_payments_from_node(node):
    df = pd.DataFrame(columns=['decimals', 'contextRef', 'unitRef', 'amount'])
    for child_node in node:
        child_node.attrib['amount'] = child_node.text
        df = df.append(child_node.attrib, ignore_index=True)

    df = df.rename(columns={'contextRef': 'reference', 'unitRef': 'currency'})
    
    return df

def get_payment_tags(node):
    """
    Obtain all the tags related with payments
    in order to do that we check that the node
    has the attribute 'unitRef' because is the
    one that has the currency of the payment
    """
    payment_tags = {}
    for child in node:
        if 'unitRef' in child.attrib:
            payment_tags[child.tag] = None
    
    return payment_tags

def format_payment_tags_with_ns(ns, payment_tags):    
    ns_key_list = list(ns.keys())
    ns_val_list = list(ns.values())

    ns_and_tag = []
    for tag in payment_tags.keys():
        current_ns, tag_end = tag.split('}')
        current_ns = current_ns[1:] # remove firts {
        position = ns_val_list.index(current_ns)
        ns_and_tag.append(f"{ns_key_list[position]}:{tag_end}")

    return ns_and_tag

In [4]:
payment_tags = get_payment_tags(root)
ns_and_tag = format_payment_tags_with_ns(ns, payment_tags)

for context in ns_and_tag:
    nodes = root.findall(f"./{context}", ns)
    df = format_payments_from_node(nodes)
    df.to_csv(f"./input/payment_{context.replace(':', '_')}.csv")

df.head()

,decimals,reference,currency,amount
0,0,IdContextosAnexo5_Transferencia1,euro,0.00
1,0,IdContextosAnexo5_Transferencia2,euro,0.00
2,0,IdContextosAnexo5_Transferencia3,euro,0.00
3,0,IdContextosAnexo5_Transferencia4,euro,95944.02
4,0,IdContextosAnexo5_Transferencia5,euro,0.00
